In [15]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [16]:
# load data

df=pd.read_csv('Cardiotocography_02_v10.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,LB,AC,FM,UC,ASTV,MSTV,ALTV,MLTV,DL,DS,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,outlier
0,146.0,0.0,0.0,5.0,65.0,0.4,33.0,7.4,0.0,0.0,...,134.0,164.0,1.0,0.0,150.0,149.0,151.0,1.0,0.0,1
1,128.0,0.0,0.0,2.0,86.0,0.3,79.0,2.9,0.0,0.0,...,114.0,130.0,0.0,0.0,128.0,126.0,129.0,0.0,1.0,1
2,149.0,0.0,0.0,5.0,61.0,0.4,34.0,5.6,0.0,0.0,...,148.0,160.0,1.0,0.0,154.0,153.0,155.0,0.0,0.0,1
3,122.0,0.0,0.0,0.0,83.0,0.5,6.0,15.6,0.0,0.0,...,62.0,130.0,0.0,0.0,122.0,122.0,123.0,3.0,1.0,1
4,134.0,0.0,4.0,0.0,79.0,0.2,42.0,5.5,0.0,0.0,...,128.0,145.0,2.0,0.0,135.0,135.0,136.0,1.0,0.0,1


In [17]:
df.shape

(1688, 22)

In [18]:
# data to values
df = df.values

In [19]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Cardiotocography.json")
gc = GCForest(config)

In [20]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [21]:
# number of class
len(np.unique(y_train))

2

In [22]:
         # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 14:42:22,332][cascade_classifier.fit_transform] X_groups_train.shape=[(1130, 21)],y_train.shape=(1130,),X_groups_test.shape=[(558, 21)],y_test.shape=(558,)
[ 2018-04-26 14:42:22,334][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-26 14:42:22,336][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:42:22,337][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-26 14:42:22,339][cascade_classifier.fit_transform] X_train.shape=(1130, 21),X_test.shape=(558, 21)
[ 2018-04-26 14:42:22,340][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1130, 21), X_cur_test.shape=(558, 21)
[ 2018-04-26 14:42:23,155][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-04-26 14:42:24,111][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.37%
[ 2018-04-26 14:42:25,289][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

In [23]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:42:32,984][cascade_classifier.transform] X_groups_test.shape=[(558, 21)]
[ 2018-04-26 14:42:32,986][cascade_classifier.transform] group_dims=[21]
[ 2018-04-26 14:42:32,988][cascade_classifier.transform] X_test.shape=(558, 21)
[ 2018-04-26 14:42:32,990][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(558, 21)


Test Accuracy of ExtraTrees = 98.566308 %


In [24]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 14:42:35,555][cascade_classifier.transform] X_groups_test.shape=[(558, 21)]
[ 2018-04-26 14:42:35,557][cascade_classifier.transform] group_dims=[21]
[ 2018-04-26 14:42:35,558][cascade_classifier.transform] X_test.shape=(558, 21)
[ 2018-04-26 14:42:35,560][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(558, 21)


In [25]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 14:42:38,060][cascade_classifier.fit_transform] X_groups_train.shape=[(1130, 23)],y_train.shape=(1130,),X_groups_test.shape=[(558, 23)],y_test.shape=(558,)
[ 2018-04-26 14:42:38,062][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-26 14:42:38,063][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:42:38,064][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-26 14:42:38,066][cascade_classifier.fit_transform] X_train.shape=(1130, 23),X_test.shape=(558, 23)
[ 2018-04-26 14:42:38,067][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1130, 23), X_cur_test.shape=(558, 23)
[ 2018-04-26 14:42:38,951][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-26 14:42:40,029][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.37%
[ 2018-04-26 14:42:41,090][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

In [26]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:42:48,858][cascade_classifier.transform] X_groups_test.shape=[(558, 23)]
[ 2018-04-26 14:42:48,860][cascade_classifier.transform] group_dims=[23]
[ 2018-04-26 14:42:48,862][cascade_classifier.transform] X_test.shape=(558, 23)
[ 2018-04-26 14:42:48,864][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(558, 23)


Test Accuracy of ExtraTrees = 99.103943 %


In [27]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [28]:
# Deep
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print ("Time",format(round(tt,3))," seconds")

[ 2018-04-26 14:42:51,399][cascade_classifier.fit_transform] X_groups_train.shape=[(1130, 23)],y_train.shape=(1130,),X_groups_test.shape=[(558, 23)],y_test.shape=(558,)
[ 2018-04-26 14:42:51,401][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-26 14:42:51,402][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:42:51,403][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-26 14:42:51,404][cascade_classifier.fit_transform] X_train.shape=(1130, 23),X_test.shape=(558, 23)
[ 2018-04-26 14:42:51,406][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1130, 23), X_cur_test.shape=(558, 23)
[ 2018-04-26 14:42:52,134][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-26 14:42:53,198][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.37%
[ 2018-04-26 14:42:54,179][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

('Test Accuracy of ExtraTrees = 99.103943 %', 0, 'fois')
('Time', '9.822', ' seconds')


[ 2018-04-26 14:43:04,663][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-26 14:43:05,797][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.37%
[ 2018-04-26 14:43:06,872][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.37%
[ 2018-04-26 14:43:07,847][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.37%
[ 2018-04-26 14:43:08,951][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.12%
[ 2018-04-26 14:43:10,030][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.35%
[ 2018-04-26 14:43:11,067][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.21%
[ 2018-04-26 14:43:12,126][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.21%


('Test Accuracy of ExtraTrees = 98.745520 %', 1, 'fois')
('Time', '10.817', ' seconds')


[ 2018-04-26 14:43:18,010][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-26 14:43:19,102][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.49%
[ 2018-04-26 14:43:20,143][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.25%
[ 2018-04-26 14:43:21,250][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.37%
[ 2018-04-26 14:43:22,390][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.12%
[ 2018-04-26 14:43:23,467][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.23%
[ 2018-04-26 14:43:24,585][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.21%
[ 2018-04-26 14:43:25,671][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.11%


...

('Test Accuracy of ExtraTrees = 98.745520 %', 17, 'fois')
('Time', '10.764', ' seconds')


[ 2018-04-26 14:46:46,868][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-26 14:46:47,822][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=93.86%
[ 2018-04-26 14:46:48,922][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.25%
[ 2018-04-26 14:46:49,907][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.25%
[ 2018-04-26 14:46:51,006][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.12%
[ 2018-04-26 14:46:52,162][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.35%
[ 2018-04-26 14:46:53,245][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.21%
[ 2018-04-26 14:46:54,350][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.21%


('Test Accuracy of ExtraTrees = 98.566308 %', 18, 'fois')
('Time', '10.635', ' seconds')


[ 2018-04-26 14:46:59,933][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-04-26 14:47:01,003][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=93.86%
[ 2018-04-26 14:47:02,107][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.37%
[ 2018-04-26 14:47:03,072][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.25%
[ 2018-04-26 14:47:04,294][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.12%
[ 2018-04-26 14:47:05,294][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.35%
[ 2018-04-26 14:47:06,307][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.21%
[ 2018-04-26 14:47:07,254][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.21%


('Test Accuracy of ExtraTrees = 98.566308 %', 19, 'fois')
('Time', '10.544', ' seconds')


# ('Test Accuracy of ExtraTrees = 99.283154 %', 1, 'fois')